In [1]:
# @title Import Dependencies

import time
import hashlib
import os
from datetime import datetime
from io import BytesIO
import random
import pyarrow
import requests
from PIL import Image
import os
import datetime as dt
import os
from datetime import datetime
import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm
from adlfs import AzureBlobFileSystem
import os
import hashlib
import os
from datetime import datetime
from io import BytesIO
import random

import requests
from PIL import Image


In [2]:
# @title Initialize Secrets

os.environ["AZURE_STORAGE_ACCOUNT_NAME"] = "ajdevreddit"

os.environ[
    "AZURE_STORAGE_ACCOUNT_KEY"] = "+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q=="

os.environ[
    "AZURE_STORE_ACCOUNT_KEY"] = "+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q=="

In [3]:
# @title TODO

class AzureFileStorageAdapter(object):
    def __init__(self, container_name: str = "data"):
        self.__account_name: str = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
        self.__account_key: str = os.environ["AZURE_STORAGE_ACCOUNT_KEY"]
        self.container_name: str = container_name

    def get_file_storage(self) -> AzureBlobFileSystem:
        return AzureBlobFileSystem(
            account_name=self.__account_name,
            account_key=self.__account_key,
            container_name=self.container_name)

    def get_file_storage_root(self) -> AzureBlobFileSystem:
        return AzureBlobFileSystem(account_name=self.__account_name, account_key=self.__account_key,
                                   container_name=self.container_name)

In [4]:
# @title TODO

tagging_schema = pyarrow.schema(
    [
        pyarrow.field("id", pyarrow.string()),
        pyarrow.field("subreddit", pyarrow.string()),
        pyarrow.field("author", pyarrow.string()),
        pyarrow.field("title", pyarrow.string()),
        pyarrow.field("caption", pyarrow.string()),
        pyarrow.field("hash", pyarrow.string()),
        pyarrow.field("permalink", pyarrow.string()),
        pyarrow.field("original_url", pyarrow.string()),
        pyarrow.field("image_name", pyarrow.string()),
        pyarrow.field("path", pyarrow.string()),
        pyarrow.field("model", pyarrow.string()),
        pyarrow.field("exists", pyarrow.bool_()),
        pyarrow.field("curated", pyarrow.bool_()),
        pyarrow.field("accept", pyarrow.bool_()),
        pyarrow.field("tags", pyarrow.list_(pyarrow.string())),
        pyarrow.field("azure_caption", pyarrow.string()),
        pyarrow.field("thumbnail_path", pyarrow.string()),
        pyarrow.field("thumbnail_exists", pyarrow.bool_()),
        pyarrow.field("thumbnail_curated", pyarrow.bool_()),
        pyarrow.field("thumbnail_accept", pyarrow.bool_()),
        pyarrow.field("additional_captions", pyarrow.list_(pyarrow.string())),
    ]
)

In [5]:
# @title TODO

tqdm.pandas(desc="Progress")

file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

In [35]:
# @title TODO

thumbnail_curation = pandas.read_parquet("data/parquet/thumbnail_curation.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)

display(thumbnail_curation)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
id,,,,,,,,,,,,,,,,,,,,,
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"[person, human face, clothing, lady, smile, ch...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"[person, human face, skin, eyelash, eyebrow, b...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"[clothing, person, human face, smile, shoulder...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"[person, human face, indoor, woman, chest, sho...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
105styc,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"[person, human face, smile, clothing, woman, i...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."


In [7]:
# @title TODO

all_data_from_captions = pandas.read_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)
all_data_from_captions.set_index('id', inplace=True, drop=False)

filtered_data_from_captions = pandas.read_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)
filtered_data_from_captions.set_index('id', inplace=True, drop=False)

display(all_data_from_captions)
display(filtered_data_from_captions)

(18789, 10)

,captionResult,objectsResult,readResult,denseCaptionsResult,modelVersion,metadata,tagsResult,smartCropsResult,peopleResult,id
id,,,,,,,,,,
10006iy,"{'confidence': 0.5599533319473267, 'text': 'a ...","{'values': [{'boundingBox': {'h': 825, 'w': 14...","{'content': 'pepsi Lite 000 AT&T STADIUM', 'mo...","{'values': [{'boundingBox': {'h': 311, 'w': 11...",2023-02-01-preview,"{'height': 5068, 'width': 7407}","{'values': [{'confidence': 0.9876041412353516,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 283, 'w': 12...",10006iy
1000cng,"{'confidence': 0.4056971073150635, 'text': 'a ...",{'values': []},"{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1768, 'w': 1...",2023-02-01-preview,"{'height': 1800, 'width': 1440}","{'values': [{'confidence': 0.9922141432762146,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...",{'values': []},1000cng
1000hda,"{'confidence': 0.6252590417861938, 'text': 'a ...","{'values': [{'boundingBox': {'h': 2233, 'w': 2...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1828, 'w': 1...",2023-02-01-preview,"{'height': 4032, 'width': 3024}","{'values': [{'confidence': 0.9986068606376648,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1824, 'w': 1...",1000hda
1000qpd,"{'confidence': 0.43016329407691956, 'text': 'a...","{'values': [{'boundingBox': {'h': 228, 'w': 56...","{'content': '@ Adam Woodworth', 'modelVersion'...","{'values': [{'boundingBox': {'h': 743, 'w': 11...",2023-02-01-preview,"{'height': 743, 'width': 1110}","{'values': [{'confidence': 0.9853595495223999,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 37, 'w': 14,...",1000qpd
1000xd6,"{'confidence': 0.4159470796585083, 'text': 'a ...","{'values': [{'boundingBox': {'h': 1761, 'w': 7...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 2268, 'w': 4...",2023-02-01-preview,"{'height': 2268, 'width': 4032}","{'values': [{'confidence': 0.9979767203330994,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1898, 'w': 8...",1000xd6


(18789, 6)

,id,captions,dense_captions,meta,tags,smart_crop
id,,,,,,
10006iy,10006iy,"[{'confidence': 0.5599533319473267, 'text': 'a...","[{'boundingBox': {'h': 311, 'w': 1196, 'x': 12...","[{'height': 5068, 'width': 7407}]","[{'confidence': 0.9876041412353516, 'name': 'g...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 458..."
1000cng,1000cng,"[{'confidence': 0.4056971073150635, 'text': 'a...","[{'boundingBox': {'h': 1768, 'w': 1209, 'x': 1...","[{'height': 1800, 'width': 1440}]","[{'confidence': 0.9922141432762146, 'name': 'b...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 136..."
1000hda,1000hda,"[{'confidence': 0.6252590417861938, 'text': 'a...","[{'boundingBox': {'h': 1828, 'w': 1186, 'x': 1...","[{'height': 4032, 'width': 3024}]","[{'confidence': 0.9986068606376648, 'name': 'f...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 280..."
1000qpd,1000qpd,"[{'confidence': 0.43016329407691956, 'text': '...","[{'boundingBox': {'h': 743, 'w': 1110, 'x': 0,...","[{'height': 743, 'width': 1110}]","[{'confidence': 0.9853595495223999, 'name': 's...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 740..."
1000xd6,1000xd6,"[{'confidence': 0.4159470796585083, 'text': 'a...","[{'boundingBox': {'h': 2268, 'w': 4032, 'x': 0...","[{'height': 2268, 'width': 4032}]","[{'confidence': 0.9979767203330994, 'name': 'o...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 225..."


In [8]:
# @title TODO

import json

display("=== Obtaining Caption Files List ===")
current_captions = file_system.ls("data/caption")
display(f"current caption files: {len(current_captions)}")

all_data = []
filtered_data = []

for caption_file in tqdm(current_captions, total=len(current_captions), desc='Reading caption files'):
    image_id = caption_file.split('/')[-1].split('.')[0]
    if image_id in all_data_from_captions['id'].values or image_id in filtered_data_from_captions['id'].values:
        continue
    try:
        file_size = file_system.size(caption_file)
        if file_size == 0:
            display(f'Empty file -- removing {image_id}', clear=True)
            file_system.rm(caption_file)
            continue
        caption_data = json.loads(file_system.read_text(caption_file, encoding='utf-8'))
        caption_data["id"] = image_id
        dense_caption_result = caption_data.get('denseCaptionsResult')
        metadata = caption_data.get('metadata')
        tags_result = caption_data.get('tagsResult')
        smart_crop_result = caption_data.get('smartCropsResult')
        basic_caption = caption_data.get('captionResult')
        _filtered_data = {
            "id": image_id,
            "captions": [basic_caption],
            "dense_captions": dense_caption_result['values'],
            "meta": [metadata],
            "tags": tags_result['values'],
            "smart_crop": smart_crop_result['values']
        }
        all_data.append(caption_data)
        filtered_data.append(_filtered_data)
    except Exception as e:
        display(f" Exception {e} for {image_id}", clear=True)
        continue

'=== Obtaining Caption Files List ==='

'current caption files: 18861'

Reading caption files: 100%|██████████| 18861/18861 [00:11<00:00, 1596.42it/s]


In [13]:
# @title TODO

new_all_data_from_captions = pandas.DataFrame(data=all_data)

new_filtered_data_from_captions = pandas.DataFrame(data=filtered_data)

display(new_all_data_from_captions)

display(new_filtered_data_from_captions)

combined_new_all_data_from_captions = pandas.concat([all_data_from_captions, new_all_data_from_captions])

combined_new_filtered_data_from_captions = pandas.concat([filtered_data_from_captions, new_filtered_data_from_captions])

display(combined_new_all_data_from_captions)

display(combined_new_filtered_data_from_captions)

combined_new_all_data_from_captions.to_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)

combined_new_filtered_data_from_captions.to_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)

,captionResult,objectsResult,readResult,denseCaptionsResult,modelVersion,metadata,tagsResult,smartCropsResult,peopleResult,id
0,"{'text': 'a woman sitting on a bed', 'confiden...","{'values': [{'boundingBox': {'x': 193, 'y': 44...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman sitting on a bed...,2023-02-01-preview,"{'width': 959, 'height': 1250}","{'values': [{'name': 'person', 'confidence': 0...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 124, 'y': 17...",13zhgjt
1,{'text': 'a woman in a garment standing on a d...,"{'values': [{'boundingBox': {'x': 417, 'y': 41...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman in a garment sta...,2023-02-01-preview,"{'width': 1080, 'height': 1350}","{'values': [{'name': 'outdoor', 'confidence': ...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 429, 'y': 41...",13zvy4w
2,{'text': 'a woman in a garment standing on a b...,"{'values': [{'boundingBox': {'x': 363, 'y': 27...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman in a garment sta...,2023-02-01-preview,"{'width': 866, 'height': 1238}","{'values': [{'name': 'outdoor', 'confidence': ...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 367, 'y': 27...",1407r7h
3,{'text': 'a woman in a garment standing in fro...,"{'values': [{'boundingBox': {'x': 338, 'y': 10...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman in a garment sta...,2023-02-01-preview,"{'width': 1080, 'height': 1288}","{'values': [{'name': 'indoor', 'confidence': 0...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 315, 'y': 13...",140ccsa
4,{'text': 'a woman in a garment standing on a d...,"{'values': [{'boundingBox': {'x': 438, 'y': 67...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman in a garment sta...,2023-02-01-preview,"{'width': 1536, 'height': 2048}","{'values': [{'name': 'outdoor', 'confidence': ...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 443, 'y': 70...",140fpxn
...,...,...,...,...,...,...,...,...,...,...
67,"{'text': 'a woman taking a selfie', 'confidenc...","{'values': [{'boundingBox': {'x': 1, 'y': 7, '...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman taking a selfie'...,2023-02-01-preview,"{'width': 1152, 'height': 2048}","{'values': [{'name': 'person', 'confidence': 0...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 0, 'y': 0, '...",145zdhi
68,"{'text': 'a woman in a garment in the water', ...","{'values': [{'boundingBox': {'x': 291, 'y': 27...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman in a garment in ...,2023-02-01-preview,"{'width': 828, 'height': 1036}","{'values': [{'name': 'outdoor', 'confidence': ...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 285, 'y': 27...",1460agf
69,{'text': 'a woman with red hair wearing a hat ...,"{'values': [{'boundingBox': {'x': 153, 'y': 11...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman with red hair we...,2023-02-01-preview,"{'width': 1284, 'height': 1804}","{'values': [{'name': 'person', 'confidence': 0...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 0, 'y': 133,...",14622g6
70,"{'text': 'a woman taking a selfie', 'confidenc...","{'values': [{'boundingBox': {'x': 20, 'y': 43,...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman taking a selfie'...,2023-02-01-preview,"{'width': 2229, 'height': 2695}","{'values': [{'name': 'person', 'confidence': 0...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 0, 'y': 0, '...",1462384


,id,captions,dense_captions,meta,tags,smart_crop
0,13zhgjt,"[{'text': 'a woman sitting on a bed', 'confide...","[{'text': 'a woman sitting on a bed', 'confide...","[{'width': 959, 'height': 1250}]","[{'name': 'person', 'confidence': 0.9979017972...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."
1,13zvy4w,[{'text': 'a woman in a garment standing on a ...,[{'text': 'a woman in a garment standing on a ...,"[{'width': 1080, 'height': 1350}]","[{'name': 'outdoor', 'confidence': 0.996706724...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."
2,1407r7h,[{'text': 'a woman in a garment standing on a ...,[{'text': 'a woman in a garment standing on a ...,"[{'width': 866, 'height': 1238}]","[{'name': 'outdoor', 'confidence': 0.992449164...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."
3,140ccsa,[{'text': 'a woman in a garment standing in fr...,[{'text': 'a woman in a garment standing in fr...,"[{'width': 1080, 'height': 1288}]","[{'name': 'indoor', 'confidence': 0.9414721727...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."
4,140fpxn,[{'text': 'a woman in a garment standing on a ...,[{'text': 'a woman in a garment standing on a ...,"[{'width': 1536, 'height': 2048}]","[{'name': 'outdoor', 'confidence': 0.999380409...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."
...,...,...,...,...,...,...
67,145zdhi,"[{'text': 'a woman taking a selfie', 'confiden...","[{'text': 'a woman taking a selfie', 'confiden...","[{'width': 1152, 'height': 2048}]","[{'name': 'person', 'confidence': 0.9995069503...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."
68,1460agf,"[{'text': 'a woman in a garment in the water',...","[{'text': 'a woman in a garment in the water',...","[{'width': 828, 'height': 1036}]","[{'name': 'outdoor', 'confidence': 0.993604838...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 78,..."
69,14622g6,[{'text': 'a woman with red hair wearing a hat...,[{'text': 'a woman with red hair wearing a hat...,"[{'width': 1284, 'height': 1804}]","[{'name': 'person', 'confidence': 0.9908113479...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."
70,1462384,"[{'text': 'a woman taking a selfie', 'confiden...","[{'text': 'a woman taking a selfie', 'confiden...","[{'width': 2229, 'height': 2695}]","[{'name': 'person', 'confidence': 0.9997074604...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."


,captionResult,objectsResult,readResult,denseCaptionsResult,modelVersion,metadata,tagsResult,smartCropsResult,peopleResult,id
0,"{'confidence': 0.5599533319473267, 'text': 'a ...","{'values': [{'boundingBox': {'h': 825, 'w': 14...","{'content': 'pepsi Lite 000 AT&T STADIUM', 'mo...","{'values': [{'boundingBox': {'h': 311, 'w': 11...",2023-02-01-preview,"{'height': 5068, 'width': 7407}","{'values': [{'confidence': 0.9876041412353516,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 283, 'w': 12...",10006iy
1,"{'confidence': 0.4056971073150635, 'text': 'a ...",{'values': []},"{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1768, 'w': 1...",2023-02-01-preview,"{'height': 1800, 'width': 1440}","{'values': [{'confidence': 0.9922141432762146,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...",{'values': []},1000cng
2,"{'confidence': 0.6252590417861938, 'text': 'a ...","{'values': [{'boundingBox': {'h': 2233, 'w': 2...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1828, 'w': 1...",2023-02-01-preview,"{'height': 4032, 'width': 3024}","{'values': [{'confidence': 0.9986068606376648,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1824, 'w': 1...",1000hda
3,"{'confidence': 0.43016329407691956, 'text': 'a...","{'values': [{'boundingBox': {'h': 228, 'w': 56...","{'content': '@ Adam Woodworth', 'modelVersion'...","{'values': [{'boundingBox': {'h': 743, 'w': 11...",2023-02-01-preview,"{'height': 743, 'width': 1110}","{'values': [{'confidence': 0.9853595495223999,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 37, 'w': 14,...",1000qpd
4,"{'confidence': 0.4159470796585083, 'text': 'a ...","{'values': [{'boundingBox': {'h': 1761, 'w': 7...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 2268, 'w': 4...",2023-02-01-preview,"{'height': 2268, 'width': 4032}","{'values': [{'confidence': 0.9979767203330994,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1898, 'w': 8...",1000xd6
...,...,...,...,...,...,...,...,...,...,...
18856,"{'text': 'a woman taking a selfie', 'confidenc...","{'values': [{'boundingBox': {'x': 1, 'y': 7, '...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman taking a selfie'...,2023-02-01-preview,"{'width': 1152, 'height': 2048}","{'values': [{'name': 'person', 'confidence': 0...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 0, 'y': 0, '...",145zdhi
18857,"{'text': 'a woman in a garment in the water', ...","{'values': [{'boundingBox': {'x': 291, 'y': 27...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman in a garment in ...,2023-02-01-preview,"{'width': 828, 'height': 1036}","{'values': [{'name': 'outdoor', 'confidence': ...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 285, 'y': 27...",1460agf
18858,{'text': 'a woman with red hair wearing a hat ...,"{'values': [{'boundingBox': {'x': 153, 'y': 11...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman with red hair we...,2023-02-01-preview,"{'width': 1284, 'height': 1804}","{'values': [{'name': 'person', 'confidence': 0...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 0, 'y': 133,...",14622g6
18859,"{'text': 'a woman taking a selfie', 'confidenc...","{'values': [{'boundingBox': {'x': 20, 'y': 43,...","{'stringIndexType': 'TextElements', 'content':...",{'values': [{'text': 'a woman taking a selfie'...,2023-02-01-preview,"{'width': 2229, 'height': 2695}","{'values': [{'name': 'person', 'confidence': 0...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'x': 0, 'y': 0, '...",1462384


,id,captions,dense_captions,meta,tags,smart_crop
0,10006iy,"[{'confidence': 0.5599533319473267, 'text': 'a...","[{'boundingBox': {'h': 311, 'w': 1196, 'x': 12...","[{'height': 5068, 'width': 7407}]","[{'confidence': 0.9876041412353516, 'name': 'g...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 458..."
1,1000cng,"[{'confidence': 0.4056971073150635, 'text': 'a...","[{'boundingBox': {'h': 1768, 'w': 1209, 'x': 1...","[{'height': 1800, 'width': 1440}]","[{'confidence': 0.9922141432762146, 'name': 'b...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 136..."
2,1000hda,"[{'confidence': 0.6252590417861938, 'text': 'a...","[{'boundingBox': {'h': 1828, 'w': 1186, 'x': 1...","[{'height': 4032, 'width': 3024}]","[{'confidence': 0.9986068606376648, 'name': 'f...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 280..."
3,1000qpd,"[{'confidence': 0.43016329407691956, 'text': '...","[{'boundingBox': {'h': 743, 'w': 1110, 'x': 0,...","[{'height': 743, 'width': 1110}]","[{'confidence': 0.9853595495223999, 'name': 's...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 740..."
4,1000xd6,"[{'confidence': 0.4159470796585083, 'text': 'a...","[{'boundingBox': {'h': 2268, 'w': 4032, 'x': 0...","[{'height': 2268, 'width': 4032}]","[{'confidence': 0.9979767203330994, 'name': 'o...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 225..."
...,...,...,...,...,...,...
18856,145zdhi,"[{'text': 'a woman taking a selfie', 'confiden...","[{'text': 'a woman taking a selfie', 'confiden...","[{'width': 1152, 'height': 2048}]","[{'name': 'person', 'confidence': 0.9995069503...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."
18857,1460agf,"[{'text': 'a woman in a garment in the water',...","[{'text': 'a woman in a garment in the water',...","[{'width': 828, 'height': 1036}]","[{'name': 'outdoor', 'confidence': 0.993604838...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 78,..."
18858,14622g6,[{'text': 'a woman with red hair wearing a hat...,[{'text': 'a woman with red hair wearing a hat...,"[{'width': 1284, 'height': 1804}]","[{'name': 'person', 'confidence': 0.9908113479...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."
18859,1462384,"[{'text': 'a woman taking a selfie', 'confiden...","[{'text': 'a woman taking a selfie', 'confiden...","[{'width': 2229, 'height': 2695}]","[{'name': 'person', 'confidence': 0.9997074604...","[{'aspectRatio': 1.0, 'boundingBox': {'x': 0, ..."


In [14]:
# @title TODO

all_data_from_captions = pandas.read_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)
display(all_data_from_captions)

filtered_data_from_captions = pandas.read_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)

display(filtered_data_from_captions)

In [15]:
# @title TODO


tags = pandas.DataFrame({'id': filtered_data_from_captions.id, 'tags': filtered_data_from_captions.tags})
out = []
for i, r in tqdm(tags.iterrows(), total=len(tags)):
    if r['tags'] is None:
        continue
    tag = pandas.json_normalize(r['tags'])
    tag['id'] = r['id']
    d = tag.to_dict(orient='records')
    out.extend(d)
converted_tags = pandas.DataFrame(data=out)
display(converted_tags)
converted_tags.to_parquet("data/parquet/tags.parquet", engine='pyarrow', filesystem=file_system)

,captionResult,objectsResult,readResult,denseCaptionsResult,modelVersion,metadata,tagsResult,smartCropsResult,peopleResult,id
0,"{'confidence': 0.5599533319473267, 'text': 'a ...","{'values': [{'boundingBox': {'h': 825, 'w': 14...","{'content': 'pepsi Lite 000 AT&T STADIUM', 'mo...","{'values': [{'boundingBox': {'h': 311, 'w': 11...",2023-02-01-preview,"{'height': 5068, 'width': 7407}","{'values': [{'confidence': 0.9876041412353516,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 283, 'w': 12...",10006iy
1,"{'confidence': 0.4056971073150635, 'text': 'a ...",{'values': []},"{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1768, 'w': 1...",2023-02-01-preview,"{'height': 1800, 'width': 1440}","{'values': [{'confidence': 0.9922141432762146,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...",{'values': []},1000cng
2,"{'confidence': 0.6252590417861938, 'text': 'a ...","{'values': [{'boundingBox': {'h': 2233, 'w': 2...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1828, 'w': 1...",2023-02-01-preview,"{'height': 4032, 'width': 3024}","{'values': [{'confidence': 0.9986068606376648,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1824, 'w': 1...",1000hda
3,"{'confidence': 0.43016329407691956, 'text': 'a...","{'values': [{'boundingBox': {'h': 228, 'w': 56...","{'content': '@ Adam Woodworth', 'modelVersion'...","{'values': [{'boundingBox': {'h': 743, 'w': 11...",2023-02-01-preview,"{'height': 743, 'width': 1110}","{'values': [{'confidence': 0.9853595495223999,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 37, 'w': 14,...",1000qpd
4,"{'confidence': 0.4159470796585083, 'text': 'a ...","{'values': [{'boundingBox': {'h': 1761, 'w': 7...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 2268, 'w': 4...",2023-02-01-preview,"{'height': 2268, 'width': 4032}","{'values': [{'confidence': 0.9979767203330994,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1898, 'w': 8...",1000xd6
...,...,...,...,...,...,...,...,...,...,...
18856,"{'confidence': 0.6667198538780212, 'text': 'a ...","{'values': [{'boundingBox': {'h': 1414, 'w': 1...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 2048, 'w': 1...",2023-02-01-preview,"{'height': 2048, 'width': 1152}","{'values': [{'confidence': 0.999506950378418, ...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 2044, 'w': 1...",145zdhi
18857,"{'confidence': 0.49799060821533203, 'text': 'a...","{'values': [{'boundingBox': {'h': 635, 'w': 46...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1036, 'w': 8...",2023-02-01-preview,"{'height': 1036, 'width': 828}","{'values': [{'confidence': 0.993604838848114, ...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 641, 'w': 48...",1460agf
18858,"{'confidence': 0.517712414264679, 'text': 'a w...","{'values': [{'boundingBox': {'h': 723, 'w': 88...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 1804, 'w': 1...",2023-02-01-preview,"{'height': 1804, 'width': 1284}","{'values': [{'confidence': 0.9908113479614258,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 1668, 'w': 1...",14622g6
18859,"{'confidence': 0.6187703013420105, 'text': 'a ...","{'values': [{'boundingBox': {'h': 2181, 'w': 2...","{'content': '', 'modelVersion': '2022-04-30', ...","{'values': [{'boundingBox': {'h': 2695, 'w': 2...",2023-02-01-preview,"{'height': 2695, 'width': 2229}","{'values': [{'confidence': 0.9997074604034424,...","{'values': [{'aspectRatio': 1.0, 'boundingBox'...","{'values': [{'boundingBox': {'h': 2691, 'w': 2...",1462384


,id,captions,dense_captions,meta,tags,smart_crop
0,10006iy,"[{'confidence': 0.5599533319473267, 'text': 'a...","[{'boundingBox': {'h': 311, 'w': 1196, 'x': 12...","[{'height': 5068, 'width': 7407}]","[{'confidence': 0.9876041412353516, 'name': 'g...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 458..."
1,1000cng,"[{'confidence': 0.4056971073150635, 'text': 'a...","[{'boundingBox': {'h': 1768, 'w': 1209, 'x': 1...","[{'height': 1800, 'width': 1440}]","[{'confidence': 0.9922141432762146, 'name': 'b...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 136..."
2,1000hda,"[{'confidence': 0.6252590417861938, 'text': 'a...","[{'boundingBox': {'h': 1828, 'w': 1186, 'x': 1...","[{'height': 4032, 'width': 3024}]","[{'confidence': 0.9986068606376648, 'name': 'f...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 280..."
3,1000qpd,"[{'confidence': 0.43016329407691956, 'text': '...","[{'boundingBox': {'h': 743, 'w': 1110, 'x': 0,...","[{'height': 743, 'width': 1110}]","[{'confidence': 0.9853595495223999, 'name': 's...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 740..."
4,1000xd6,"[{'confidence': 0.4159470796585083, 'text': 'a...","[{'boundingBox': {'h': 2268, 'w': 4032, 'x': 0...","[{'height': 2268, 'width': 4032}]","[{'confidence': 0.9979767203330994, 'name': 'o...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 225..."
...,...,...,...,...,...,...
18856,145zdhi,"[{'confidence': 0.6667198538780212, 'text': 'a...","[{'boundingBox': {'h': 2048, 'w': 1152, 'x': 0...","[{'height': 2048, 'width': 1152}]","[{'confidence': 0.999506950378418, 'name': 'pe...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 106..."
18857,1460agf,"[{'confidence': 0.49799060821533203, 'text': '...","[{'boundingBox': {'h': 1036, 'w': 828, 'x': 0,...","[{'height': 1036, 'width': 828}]","[{'confidence': 0.993604838848114, 'name': 'ou...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 689..."
18858,14622g6,"[{'confidence': 0.517712414264679, 'text': 'a ...","[{'boundingBox': {'h': 1804, 'w': 1284, 'x': 0...","[{'height': 1804, 'width': 1284}]","[{'confidence': 0.9908113479614258, 'name': 'p...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 127..."
18859,1462384,"[{'confidence': 0.6187703013420105, 'text': 'a...","[{'boundingBox': {'h': 2695, 'w': 2229, 'x': 0...","[{'height': 2695, 'width': 2229}]","[{'confidence': 0.9997074604034424, 'name': 'p...","[{'aspectRatio': 1.0, 'boundingBox': {'h': 180..."


In [16]:
# @title TODO

dense = pandas.DataFrame({'id': filtered_data_from_captions.id, 'dense_captions': filtered_data_from_captions.dense_captions})

out = []
for i, r in tqdm(dense.iterrows(), total=len(dense)):
    if r['dense_captions'] is None:
        continue
    dense_caption = pandas.json_normalize(r['dense_captions'])
    dense_caption['id'] = r['id']
    d = dense_caption.to_dict(orient='records')
    out.extend(d)

converted = pandas.DataFrame(data=out)
display(converted)
converted.to_parquet("data/parquet/dense_captions.parquet", engine='pyarrow', filesystem=file_system)

100%|██████████| 18861/18861 [00:16<00:00, 1146.08it/s]


,confidence,name,id
0,0.987604,grass,10006iy
1,0.923687,outdoor,10006iy
2,0.915661,artificial turf,10006iy
3,0.912514,stadium,10006iy
4,0.880675,sports equipment,10006iy
...,...,...,...
279003,0.854841,abdomen,1464c8z
279004,0.850876,photo shoot,1464c8z
279005,0.830261,swimsuit,1464c8z
279006,0.826516,girl,1464c8z


In [17]:
# @title TODO

single_caption_data = pandas.DataFrame(
    {
        'id': all_data_from_captions['id'],
        'azure_caption': [item['text'] for item in all_data_from_captions['captionResult']],
        'tags': [[foo['name'] for foo in item['values']] for item in all_data_from_captions['tagsResult']]
    })

single_caption_data_indexed = single_caption_data.set_index("id")

for index, row in tqdm(single_caption_data_indexed.iterrows(), total=len(single_caption_data_indexed)):
    print(index)

display("== Updated With Basic Captions ==")
display(thumbnail_curation)

100%|██████████| 18861/18861 [00:21<00:00, 870.09it/s]


,confidence,text,boundingBox.h,boundingBox.w,boundingBox.x,boundingBox.y,id
0,0.481536,a close-up of a sign,311,1196,1209,1542,10006iy
1,0.485781,a football helmet on grass,1853,7329,0,3068,10006iy
2,0.559779,a lizard on a helmet on a football field,5068,7407,0,0,10006iy
3,0.519275,a lizard on a ball,935,1461,2609,2192,10006iy
4,0.660839,a lizard on a football helmet,2190,2613,2380,2685,10006iy
...,...,...,...,...,...,...,...
156105,0.577244,a woman with sunglasses on her head,704,1498,0,0,1464c8z
156106,0.439448,a red ribbon tied around a woman's waist,116,112,833,1239,1464c8z
156107,0.488155,a woman lying on the sand,858,1496,0,1163,1464c8z
156108,0.502893,a person's hand on a beach,183,190,609,812,1464c8z


In [43]:
# @title TODO

%%time



accepted_final.to_parquet("data/parquet/curation_2.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)
del accepted_final

,id,azure_caption,tags
id,,,
10006iy,10006iy,a lizard on a helmet on a football field,"[grass, outdoor, artificial turf, stadium, spo..."
1000cng,1000cng,a black and white photo of a hallway with arches,"[black and white, building, architecture, colu..."
1000hda,1000hda,a hand holding a red and white flower,"[flower, plant, petal, poppy, annual plant, ou..."
1000qpd,1000qpd,a boat on the shore with green and purple ligh...,"[sky, boat, outdoor, watercraft, cloud, nature..."
1000xd6,1000xd6,a woman painting a picture on a beach,"[outdoor, clothing, plant, sky, beach, tree, g..."
...,...,...,...
145zdhi,145zdhi,a woman taking a selfie,"[person, human face, smile, clothing, stomach,..."
1460agf,1460agf,a woman in a garment in the water,"[outdoor, sky, person, water, beach, swimwear,..."
14622g6,14622g6,a woman with red hair wearing a hat and holdin...,"[person, human face, clothing, fashion accesso..."


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
id,,,,,,,,,,,,,,,,,,,,,
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"[person, human face, clothing, lady, smile, ch...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"[person, human face, skin, eyelash, eyebrow, b...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"[clothing, person, human face, smile, shoulder...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"[person, human face, indoor, woman, chest, sho...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
105styc,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"[person, human face, smile, clothing, woman, i...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145tbof,145tbof,SFWNextDoorGirls,Significant_Metal561,What do you think of my curly hair neighbor :>,arafed woman with red hair wearing a blue shir...,02ab285b328cc773effab24fdcb579bb,/r/SFWNextDoorGirls/comments/145tbof/what_do_y...,https://i.redd.it/9oqk53om455b1.jpg,145tbof.jpg,data/image/145tbof.jpg,...,True,True,True,[],,,False,False,False,[]
145xl1f,145xl1f,SFWNextDoorGirls,Rex-Costanza,Good european beach day!!,there is a woman in a bikini standing in the w...,568f76e4a155cc836533b804f53d6bb8,/r/SFWNextDoorGirls/comments/145xl1f/good_euro...,https://i.imgur.com/CUFysDF.jpg,145xl1f.jpg,data/image/145xl1f.jpg,...,True,True,True,[],,,False,False,False,[]
145xxte,145xxte,SFWNextDoorGirls,innerworldxoxo,I lie and look at you,there is a woman wearing glasses laying on a c...,64b70c90d333cce9c5bc8572c4c493bf,/r/SFWNextDoorGirls/comments/145xxte/i_lie_and...,https://i.redd.it/ts4ss5tif65b1.jpg,145xxte.jpg,data/image/145xxte.jpg,...,True,True,True,[],,,False,False,False,[]


100%|██████████| 18831/18831 [00:19<00:00, 977.81it/s] 


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
id,,,,,,,,,,,,,,,,,,,,,
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"person,human face,clothing,lady,smile,chest,wo...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"person,human face,skin,eyelash,eyebrow,brown h...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"clothing,person,human face,smile,shoulder,elbo...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"person,human face,indoor,clothing,woman,chest,...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
105styc,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"person,human face,smile,clothing,woman,indoor,...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145tbof,145tbof,SFWNextDoorGirls,Significant_Metal561,What do you think of my curly hair neighbor :>,arafed woman with red hair wearing a blue shir...,02ab285b328cc773effab24fdcb579bb,/r/SFWNextDoorGirls/comments/145tbof/what_do_y...,https://i.redd.it/9oqk53om455b1.jpg,145tbof.jpg,data/image/145tbof.jpg,...,True,True,True,"person,clothing,indoor,human face,wall,red hai...",a woman with red hair and a scarf,,False,False,False,[]
145xl1f,145xl1f,SFWNextDoorGirls,Rex-Costanza,Good european beach day!!,there is a woman in a bikini standing in the w...,568f76e4a155cc836533b804f53d6bb8,/r/SFWNextDoorGirls/comments/145xl1f/good_euro...,https://i.imgur.com/CUFysDF.jpg,145xl1f.jpg,data/image/145xl1f.jpg,...,True,True,True,"outdoor,person,sky,swimwear,human face,water,c...",a woman in a bathing suit,,False,False,False,[]
145xxte,145xxte,SFWNextDoorGirls,innerworldxoxo,I lie and look at you,there is a woman wearing glasses laying on a c...,64b70c90d333cce9c5bc8572c4c493bf,/r/SFWNextDoorGirls/comments/145xxte/i_lie_and...,https://i.redd.it/ts4ss5tif65b1.jpg,145xxte.jpg,data/image/145xxte.jpg,...,True,True,True,"person,human face,girl,model,indoor,black hair...",a woman wearing glasses and a garment lying on...,,False,False,False,[]


In [45]:
# @title TODO

%%time



accepted_final = pandas.read_parquet("data/parquet/curation_2.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)
display(accepted_final)

In [47]:
# @title TODO

%%time



def get_aspect_ratio(x: object):
    return x['crops'][0]['aspectRatio']


def get_bounding_box(x: object):
    return x['crops'][0]['boundingBox']

Progress: 100%|██████████| 18861/18861 [00:00<00:00, 160826.14it/s]


,id,crops,aspectRatio,bounding_box,x,y,w,h
id,,,,,,,,
10006iy,10006iy,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 458...",1.0,"{'h': 4585, 'w': 4586, 'x': 1157, 'y': 475}",1157,475,4586,4585
1000cng,1000cng,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 136...",1.0,"{'h': 1367, 'w': 1367, 'x': 0, 'y': 169}",0,169,1367,1367
1000hda,1000hda,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 280...",1.0,"{'h': 2800, 'w': 2799, 'x': 142, 'y': 550}",142,550,2799,2800
1000qpd,1000qpd,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 740...",1.0,"{'h': 740, 'w': 741, 'x': 173, 'y': 0}",173,0,741,740
1000xd6,1000xd6,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 225...",1.0,"{'h': 2259, 'w': 2259, 'x': 702, 'y': 0}",702,0,2259,2259
...,...,...,...,...,...,...,...,...
145zdhi,145zdhi,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 106...",1.0,"{'h': 1066, 'w': 1066, 'x': 0, 'y': 0}",0,0,1066,1066
1460agf,1460agf,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 689...",1.0,"{'h': 689, 'w': 689, 'x': 78, 'y': 155}",78,155,689,689
14622g6,14622g6,"[{'aspectRatio': 1.0, 'boundingBox': {'h': 127...",1.0,"{'h': 1279, 'w': 1279, 'x': 0, 'y': 123}",0,123,1279,1279


In [48]:
# @title TODO

%%time


cropping = pandas.DataFrame(
    {'id': filtered_data_from_captions['id'], 'crops': filtered_data_from_captions['smart_crop']}).set_index('id',
                                                                                                             drop=False)

cropping['aspectRatio'] = cropping.progress_apply(lambda x: get_aspect_ratio(x), axis=1)
cropping['bounding_box'] = cropping.progress_apply(lambda x: get_bounding_box(x), axis=1)
cropping['x'] = cropping.progress_apply(lambda x: x['bounding_box']['x'], axis=1)
cropping['y'] = cropping.progress_apply(lambda x: x['bounding_box']['y'], axis=1)
cropping['w'] = cropping.progress_apply(lambda x: x['bounding_box']['w'], axis=1)
cropping['h'] = cropping.progress_apply(lambda x: x['bounding_box']['h'], axis=1)

display(cropping)

In [49]:
# @title TODO

%%time


def create_thumbnail(target_image_id: str, file_names: list, crops: pandas.DataFrame, curated_data: pandas.DataFrame):
    _file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

    out_path = f"data/image/thumbnail/{target_image_id}.jpg"
    try:
        if target_image_id is None or out_path in file_names:
            # print(f'Image {target_image_id} already exists, skipping')
            return out_path

        cropping_information = cropping.loc[crops['id'] == target_image_id]
        if cropping_information is None or len(cropping_information) == 0:
            # print(f'No cropping information for {target_image_id}, skipping')
            return "/data/nope"

        record = curated_data.loc[curated_data['id'] == target_image_id]
        record_path = record.to_dict(orient='records')[0]['path']
        image_url = file_system.url(record_path)
        original_image = Image.open(requests.get(image_url, stream=True).raw)
        copied_image = original_image.copy()
        original_image.close()

        cropped = copied_image.crop((cropping_information['x'].values[0],
                                     cropping_information['y'].values[0],
                                     cropping_information['x'].values[0] +
                                     cropping_information['w'].values[0],
                                     cropping_information['y'].values[0] +
                                     cropping_information['h'].values[0]))
        copied_image.close()

        resized = cropped.resize((512, 512), 1)
        resized.save('temp.jpg')
        resized.close()
        file_system.upload('temp.jpg', out_path, overwrite=True)
        print(f'Thumbnail created for {target_image_id}')
        return out_path

    except Exception as ex:
        print(f'Error creating thumbnail for {target_image_id}: {ex}')
        return "/data/nope"

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
id,,,,,,,,,,,,,,,,,,,,,
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"person,human face,clothing,lady,smile,chest,wo...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"person,human face,skin,eyelash,eyebrow,brown h...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"clothing,person,human face,smile,shoulder,elbo...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"person,human face,indoor,clothing,woman,chest,...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
105styc,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"person,human face,smile,clothing,woman,indoor,...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145tbof,145tbof,SFWNextDoorGirls,Significant_Metal561,What do you think of my curly hair neighbor :>,arafed woman with red hair wearing a blue shir...,02ab285b328cc773effab24fdcb579bb,/r/SFWNextDoorGirls/comments/145tbof/what_do_y...,https://i.redd.it/9oqk53om455b1.jpg,145tbof.jpg,data/image/145tbof.jpg,...,True,True,True,"person,clothing,indoor,human face,wall,red hai...",a woman with red hair and a scarf,,False,False,False,[]
145xl1f,145xl1f,SFWNextDoorGirls,Rex-Costanza,Good european beach day!!,there is a woman in a bikini standing in the w...,568f76e4a155cc836533b804f53d6bb8,/r/SFWNextDoorGirls/comments/145xl1f/good_euro...,https://i.imgur.com/CUFysDF.jpg,145xl1f.jpg,data/image/145xl1f.jpg,...,True,True,True,"outdoor,person,sky,swimwear,human face,water,c...",a woman in a bathing suit,,False,False,False,[]
145xxte,145xxte,SFWNextDoorGirls,innerworldxoxo,I lie and look at you,there is a woman wearing glasses laying on a c...,64b70c90d333cce9c5bc8572c4c493bf,/r/SFWNextDoorGirls/comments/145xxte/i_lie_and...,https://i.redd.it/ts4ss5tif65b1.jpg,145xxte.jpg,data/image/145xxte.jpg,...,True,True,True,"person,human face,girl,model,indoor,black hair...",a woman wearing glasses and a garment lying on...,,False,False,False,[]


In [62]:
# @title TODO

%%time


file_names = file_system.ls('data/image/thumbnail')

accepted_final['thumbnail_path'] = accepted_final.progress_apply(
    lambda x: create_thumbnail(x['id'], file_names, cropping, accepted_final), axis=1)

display(accepted_final)

100%|██████████| 18831/18831 [00:00<00:00, 22310.39it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
0,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"person,human face,clothing,lady,smile,chest,wo...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
1,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"person,human face,skin,eyelash,eyebrow,brown h...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
2,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"clothing,person,human face,smile,shoulder,elbo...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
3,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"person,human face,indoor,clothing,woman,chest,...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
4,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"person,human face,smile,clothing,woman,indoor,...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18826,145tbof,SFWNextDoorGirls,Significant_Metal561,What do you think of my curly hair neighbor :>,arafed woman with red hair wearing a blue shir...,02ab285b328cc773effab24fdcb579bb,/r/SFWNextDoorGirls/comments/145tbof/what_do_y...,https://i.redd.it/9oqk53om455b1.jpg,145tbof.jpg,data/image/145tbof.jpg,...,True,True,True,"person,clothing,indoor,human face,wall,red hai...",a woman with red hair and a scarf,data/image/thumbnail/145tbof.jpg,True,False,False,[]
18827,145xl1f,SFWNextDoorGirls,Rex-Costanza,Good european beach day!!,there is a woman in a bikini standing in the w...,568f76e4a155cc836533b804f53d6bb8,/r/SFWNextDoorGirls/comments/145xl1f/good_euro...,https://i.imgur.com/CUFysDF.jpg,145xl1f.jpg,data/image/145xl1f.jpg,...,True,True,True,"outdoor,person,sky,swimwear,human face,water,c...",a woman in a bathing suit,data/image/thumbnail/145xl1f.jpg,True,False,False,[]
18828,145xxte,SFWNextDoorGirls,innerworldxoxo,I lie and look at you,there is a woman wearing glasses laying on a c...,64b70c90d333cce9c5bc8572c4c493bf,/r/SFWNextDoorGirls/comments/145xxte/i_lie_and...,https://i.redd.it/ts4ss5tif65b1.jpg,145xxte.jpg,data/image/145xxte.jpg,...,True,True,True,"person,human face,girl,model,indoor,black hair...",a woman wearing glasses and a garment lying on...,data/image/thumbnail/145xxte.jpg,True,False,False,[]
18829,145ykai,SFWNextDoorGirls,sweetchloejane,"Just wanted to wish you a good day, neig

In [63]:
# @title TODO

%%time



accepted_final['thumbnail_exists'] = accepted_final.progress_apply(lambda x: file_system.exists(x['thumbnail_path']),
                                                                   axis=1)

display(accepted_final)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
0,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,...,True,True,True,"person,human face,clothing,lady,smile,chest,wo...",a woman taking a selfie,data/image/thumbnail/1013bdt.jpg,True,True,True,"[a woman taking a selfie, a close up of an eye..."
1,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,...,True,True,True,"person,human face,skin,eyelash,eyebrow,brown h...",a woman taking a selfie,data/image/thumbnail/105mekt.jpg,True,True,True,"[a close up of a person's eye, a woman taking ..."
2,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,...,True,True,True,"clothing,person,human face,smile,shoulder,elbo...",a woman smiling at camera,data/image/thumbnail/105qvgl.jpg,True,True,True,"[a woman smiling at camera, a woman with long ..."
3,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,...,True,True,True,"person,human face,indoor,clothing,woman,chest,...",a woman taking a selfie,data/image/thumbnail/105rpcj.jpg,True,True,True,"[a woman taking a selfie, close up of a person..."
4,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,...,True,True,True,"person,human face,smile,clothing,woman,indoor,...",a woman taking a selfie,data/image/thumbnail/105styc.jpg,True,True,True,"[a woman smiling at the camera, close-up of a ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18826,145tbof,SFWNextDoorGirls,Significant_Metal561,What do you think of my curly hair neighbor :>,arafed woman with red hair wearing a blue shir...,02ab285b328cc773effab24fdcb579bb,/r/SFWNextDoorGirls/comments/145tbof/what_do_y...,https://i.redd.it/9oqk53om455b1.jpg,145tbof.jpg,data/image/145tbof.jpg,...,True,True,True,"person,clothing,indoor,human face,wall,red hai...",a woman with red hair and a scarf,data/image/thumbnail/145tbof.jpg,True,False,False,[]
18827,145xl1f,SFWNextDoorGirls,Rex-Costanza,Good european beach day!!,there is a woman in a bikini standing in the w...,568f76e4a155cc836533b804f53d6bb8,/r/SFWNextDoorGirls/comments/145xl1f/good_euro...,https://i.imgur.com/CUFysDF.jpg,145xl1f.jpg,data/image/145xl1f.jpg,...,True,True,True,"outdoor,person,sky,swimwear,human face,water,c...",a woman in a bathing suit,data/image/thumbnail/145xl1f.jpg,True,False,False,[]
18828,145xxte,SFWNextDoorGirls,innerworldxoxo,I lie and look at you,there is a woman wearing glasses laying on a c...,64b70c90d333cce9c5bc8572c4c493bf,/r/SFWNextDoorGirls/comments/145xxte/i_lie_and...,https://i.redd.it/ts4ss5tif65b1.jpg,145xxte.jpg,data/image/145xxte.jpg,...,True,True,True,"person,human face,girl,model,indoor,black hair...",a woman wearing glasses and a garment lying on...,data/image/thumbnail/145xxte.jpg,True,False,False,[]
18829,145ykai,SFWNextDoorGirls,sweetchloejane,"Just wanted to wish you a good day, neig

In [67]:
# @title TODO

%%time



accepted_copy = accepted_final.copy()

accepted_copy['additional_captions'] = accepted_copy.progress_apply(lambda x: [], axis=1)

accepted_slice = accepted_copy.loc[accepted_copy['thumbnail_exists'] == True, tagging_schema.names]

accepted_slice.dropna(inplace=True)

accepted_slice.reset_index(inplace=True, drop=True)

display(accepted_slice)

In [68]:
# @title TODO

%%time


current_data.set_index('id', inplace=True, drop=False)
accepted_slice.set_index('id', inplace=True, drop=False)

filtered = accepted_slice.loc(accepted_slice['id'].isnotin(current_data['id']))
filtered.dropna(inplace=True)
filtered.reset_index(inplace=True, drop=True)
display(filtered)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,...,exists,curated,accept,tags,azure_caption,thumbnail_path,thumbnail_exists,thumbnail_curated,thumbnail_accept,additional_captions
18830,145zdhi,SFWNextDoorGirls,miss_violette_,Felt cutesy in my new top for a hot June day,arafed woman in blue tank top and denim shorts...,f929c1d08bf6c1bccbf429f731af6af1,/r/SFWNextDoorGirls/comments/145zdhi/felt_cute...,http://i.imgur.com/Rz7Boxf.jpg,145zdhi.jpg,data/image/145zdhi.jpg,...,True,True,True,"[p, e, r, s, o, n, ,, h, u, m, a, n, , f, a, ...",a woman taking a selfie,data/image/thumbnail/145zdhi.jpg,True,False,False,[]
